<a href="https://www.kaggle.com/code/hopesb/backpack-baseline-predicition?scriptVersionId=223229280" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Backpack Predition

In [1]:
%load_ext cudf.pandas

In [2]:
# import all necessary libraries.
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px

import sklearn
from sklearn.metrics import mean_squared_error
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from tqdm.notebook import tqdm
import optuna
import warnings
warnings.simplefilter("ignore")

In [3]:
%%time
# Reading the files
test_filepath = "/kaggle/input/playground-series-s5e2/test.csv"
train_filepath = "/kaggle/input/playground-series-s5e2/train.csv"
df = pd.read_csv(train_filepath, index_col="id")
df_test = pd.read_csv(test_filepath, index_col="id")

CPU times: user 64.6 ms, sys: 145 ms, total: 210 ms
Wall time: 410 ms


In [4]:
%%time
extra_filepath = "/kaggle/input/playground-series-s5e2/training_extra.csv"
df_extra = pd.read_csv(extra_filepath, index_col="id")
df_extra.shape

CPU times: user 187 ms, sys: 185 ms, total: 372 ms
Wall time: 1.05 s


(3694318, 10)

## Explore

In [5]:
print(df.shape)
df.head()

(300000, 10)


,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [6]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   Brand                 290295 non-null  object
 1   Material              291653 non-null  object
 2   Size                  293405 non-null  object
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object
 5   Waterproof            292950 non-null  object
 6   Style                 292030 non-null  object
 7   Color                 290050 non-null  object
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 27.2+ MB


In [7]:
df.isnull().sum().sort_values()

Compartments               0
Price                      0
Weight Capacity (kg)     138
Size                    6595
Waterproof              7050
Laptop Compartment      7444
Style                   7970
Material                8347
Brand                   9705
Color                   9950
dtype: int64

In [8]:
df.nunique().sort_values()

Laptop Compartment           2
Waterproof                   2
Size                         3
Style                        3
Material                     4
Brand                        5
Color                        6
Compartments                10
Price                    48212
Weight Capacity (kg)    181598
dtype: int64

In [9]:
df.describe()

,Compartments,Weight Capacity (kg),Price
count,300000.000000,299862.000000,300000.000000
mean,5.443590,18.029994,81.411107
std,2.890766,6.966914,39.039340
min,1.000000,5.000000,15.000000
25%,3.000000,12.097867,47.384620
50%,5.000000,18.068614,80.956120
75%,8.000000,24.002375,115.018160
max,10.000000,30.000000,150.000000


In [10]:
def wrangle(filepaths, train=True):
    if train: 
        df_1 = pd.read_csv(filepaths[0], index_col= "id")
        df_2 = pd.read_csv(filepaths[-1], index_col="id")
        # Merge the dataset.
        df = pd.concat([df_1, df_2])
    else:
        df = pd.read_csv(filepaths, index_col="id")

    
    # Fill the weight with mean.
    df["Weight Capacity (kg)"] = df["Weight Capacity (kg)"].fillna(df["Weight Capacity (kg)"].mean())
    # fill the color.
    df["Color"] = df["Color"].fillna("Multi-color")
    # fill the Brand with unknown brand.
    df["Brand"] = df["Brand"].fillna("Unknown")
    # fill the size.
    df["Size"] = df["Size"].fillna("Others")
    # fill the waterproof.
    df["Waterproof"] = df["Waterproof"].fillna("Undetermined")
    # fill the Laptop compartment.
    df["Laptop Compartment"] = df["Laptop Compartment"].fillna("Unknown")
    # Fill the material ans style.
    df["Material"] = df["Material"].fillna("Unknown")
    df["Style"] = df["Style"].fillna("Unknown")

    df["Compartments"] = df["Compartments"].astype(int)
    df["Weight Capacity (kg)"] = round(df["Weight Capacity (kg)"], 2)
    df["Weight Class"] = df["Weight Capacity (kg)"].apply(lambda x: "Minimum" if x < 10 else(
        "Above_Minimum" if x < 15 else(
            "Medium" if x < 25 else "High"
        )
    ))

    ordinal = OrdinalEncoder()
    df = ordinal.fit_transform(df)

    return df

In [11]:
%%time
filepaths = [train_filepath, extra_filepath]
df = wrangle(filepaths)
df.isnull().sum()

CPU times: user 11.7 s, sys: 1.15 s, total: 12.8 s
Wall time: 13.2 s


Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
Weight Class            0
dtype: int64

In [12]:
df.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,Weight Class
id,,,,,,,,,,,
0,1,1,1,7,1,1,1,1,11.61,112.15875,1
1,1,2,2,10,1,2,2,2,27.08,68.88056,2
2,2,1,2,2,1,1,2,3,16.64,39.17320,3
3,3,3,2,8,1,1,2,2,12.94,80.60793,1
4,4,2,1,1,1,2,2,2,17.75,86.02312,3


## Split Dataset.

In [13]:
def target_encode(df_train, encode_col, df_test = None, target="Price", eps=0):
    for col in encode_col:
        mean = df_train[target].mean()
        agg_count = df_train[[col, target]].groupby(col).count().reset_index()
        agg_sum = df_train[[col, target]].groupby(col).sum().reset_index()
        agg_sum = agg_sum.merge(agg_count, on=col, suffixes=("_sum", "_count"))
        eps = eps * len(agg_sum)
        agg_sum[f"{col}_tgt_enc"] = (agg_sum[f"{target}_sum"] + eps * mean) / (agg_sum[f"{target}_count"] + eps)
        encode = agg_sum[[col, f"{col}_tgt_enc"]]
        df_train = df_train.merge(encode, on=col)
        if df_test is not None:
            df_test= df_test.merge(encode, on=col)
    df_train.drop(columns=encode_col, inplace=True)
    df_test.drop(columns=encode_col, inplace=True)
    return df_train, df_test

In [14]:
test_df = wrangle(test_filepath, train=False)

In [15]:
enc_col = ["Brand", "Material", "Laptop Compartment", "Waterproof", "Style", "Weight Class"]
train_enc, test_enc = target_encode(df, enc_col, test_df, eps=50)
train_enc

,Size,Compartments,Color,Weight Capacity (kg),Price,Brand_tgt_enc,Material_tgt_enc,Laptop Compartment_tgt_enc,Waterproof_tgt_enc,Style_tgt_enc,Weight Class_tgt_enc
0,1,7,1,11.61,112.15875,81.777383,80.490089,81.420058,81.437994,81.494502,81.568881
1,2,10,2,27.08,68.88056,81.777383,81.830245,81.420058,81.411091,81.192425,81.782117
2,2,2,3,16.64,39.17320,82.064944,80.490089,81.420058,81.437994,81.192425,81.732499
3,2,8,2,12.94,80.60793,81.284835,81.072255,81.420058,81.437994,81.192425,81.568881
4,1,1,2,17.75,86.02312,80.527997,81.830245,81.420058,81.411091,81.192425,81.732499
...,...,...,...,...,...,...,...,...,...,...,...
3994313,4,3,4,28.10,104.74460,81.284835,81.830245,81.420058,81.411091,81.192425,81.782117
3994314,2,10,4,17.38,122.39043,81.225631,80.490089,81.420058,81.411091,81.494502,81.732499
3994315,3,10,3,17.04,148.18470,81.777383,81.830245,81.350514,81.437994,81.357899,81.732499
3994316,4,2,5,28.78,22.32269,81.225631,81.830245,81.350514,81.437994,81.357899,81.782117


In [16]:
target = "Price"
X = train_enc.drop(columns= target)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model.

In [17]:
# Create a function that takes in a list of algortithm and then make the prediction then save the prediction.
def score_predict(algorithms, X_train, X_test, y_train, y_test, test_enc, test_df=None, save=True): 

    data_dict = {}
    # loop throught the algorithms.
    for algorithm in tqdm(algorithms, desc="Starting"):
        # make the pipeline
        model = make_pipeline(
            StandardScaler(),
            algorithm
        )
        # Fitting the model.
        alg_name = list(model.named_steps.keys())[-1]
        print(f"Fitting the {alg_name}")
        model.fit(X_train, y_train)
        # Get the score.
        print("Making Prediction")
        y_test_pred = model.predict(X_test)
        score = np.sqrt(mean_squared_error(y_test, y_test_pred))
        data_dict[alg_name] = score

        if save == True:
            y_pred = model.predict(test_enc)
            pd.DataFrame({"Price": y_pred}, index=test_df.index).to_csv(f"{alg_name}.csv")

    score_df = pd.DataFrame(data_dict, index=["Accuracy"]).T
    return score_df

In [18]:
xgb_params = {'max_depth': 8, 
              'n_estimators': 1701, 
              'learning_rate': 0.006899755927601937, 
              'min_child_weight': 0.2751236938771351, 
              'reg_alpha': 0.527480714707444, 
              'reg_lambda': 13.198806851176458, 
              'colsample_bylevel': 0.9772481366672879, 
              'colsample_bytree': 0.7774789504388184, 
              'colsample_bynode': 0.8541180728630634}
lgb_params = {'max_depth': 17, 
              'n_estimators': 2995, 
              'learning_rate': 0.00943875220877273, 
              'min_child_weight': 0.9260630565539196, 
              'reg_alpha': 0.9200603427801367, 
              'reg_lambda': 1.4831257707229202, 
              'colsample_bylevel': 0.5223344478716733, 
              'colsample_bytree': 0.9937966786732069, 
              'colsample_bynode': 0.8479188035499157}
cat_params = {'max_depth': 8, 
              'n_estimators': 2830, 
              'learning_rate': 0.009960055354186655}
algorithms = [LGBMRegressor(**lgb_params, verbose=-1, random_state=42),
             XGBRegressor(**xgb_params, random_state=42),
             CatBoostRegressor(**cat_params, verbose=0, random_state=42)]
score = score_predict(algorithms, X_train, X_test, y_train, y_test, test_enc, test_df)
score

Starting:   0%|          | 0/3 [00:00<?, ?it/s]

Fitting the lgbmregressor
Making Prediction
Fitting the xgbregressor
Making Prediction
Fitting the catboostregressor
Making Prediction


,Accuracy
lgbmregressor,38.849105
xgbregressor,38.852445
catboostregressor,38.850459


In [19]:
lgb = make_pipeline(
    StandardScaler(),
    LGBMRegressor(**lgb_params, verbose=-1, random_state=42)
)
cat = make_pipeline(
    StandardScaler(),
    CatBoostRegressor(**cat_params, verbose=0, random_state=42)
)
xgb = make_pipeline(
    StandardScaler(),
    XGBRegressor(**xgb_params, random_state=42)
)

estimators = [("lgb", lgb), ("cat", cat), ("xgb", xgb)]
vote = VotingRegressor(estimators=estimators, weights=[3, 2, 1])
vote.fit(X_train, y_train)
y_pred = vote.predict(test_enc)
pd.DataFrame({"Price": y_pred}, index=test_df.index).to_csv(f"submission.csv")